In [9]:
### This notebook will be used to complete the capstone project for the IBM Data Science Professional Certificate.

In [8]:
import pandas as pd
import numpy as np

In [7]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
